# Reduced Model attempt 15: Lumped Parameters
In this notebook, we will take a look at red mod 5 and try to see if we can automate a way to find the lumped parameters that frequently show up.

In [ ]:
%run sbml_to_ode2.py

In [ ]:
from auto_reduce import *
from auto_reduce.utils import get_ODE
import matplotlib.pyplot as plt
from auto_reduce.utils import reduce

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()
from bokeh.layouts import row

import colorcet

from sympy import *

In [3]:
filename = 'CRN.xml'
x,f,P,params_values,x_init = sbml_to_ode2(filename)

nouts = 1
n = len(x)
# Make ATP main output
C = np.zeros((nouts, n), dtype = int)
C[0][1] = 1 
C = C.tolist()

# Setup system
timepoints_ode = np.linspace(0,75,100)

error_tol = 100
nstates_tol = 1

sys = System(x, f, params = P, params_values = params_values, C = C, x_init = x_init)

sys_ode = get_ODE(sys, timepoints_ode)
sol = sys_ode.solve_system()

In [4]:
species_to_remove = [5,6,9,10,13, 14,17,18,23,24,28,29,31,32,
                 35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64]

timescale_list = []
for i in species_to_remove:
    timescale_list.append(x[i])

# Get appropriate list of complexes that are not in timescale_list
# What you should pass into solve_timescale_separation
update_timescale = []
for item in x:
    if item not in timescale_list:
        update_timescale.append(item)    
#update_timescale

In [5]:
timepoints_ssm = np.linspace(0,75,100)
timepoints_ode = np.linspace(0, 75, 100)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm)

aa,bb = sys_reduce.solve_timescale_separation(update_timescale)
print('the ATP error is:',sys_reduce.get_error_metric(aa))

sys_ode = get_ODE(aa, timepoints_ode)
sol = sys_ode.solve_system()

attempting to retain : [0, 1, 2, 3, 4, 7, 8, 11, 12, 15, 16, 19, 20, 21, 22, 25, 26, 27, 30, 33, 34, 37, 38, 41, 42, 45, 46, 49, 50, 53, 54, 57, 58, 61, 62, 65, 66, 67]


/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/sympy/core/relational.py:470: SymPyDeprecationWarning: 

Eq(expr) with rhs default to 0 has been deprecated since SymPy 1.5.
Use Eq(expr, 0) instead. See
https://github.com/sympy/sympy/issues/16587 for more info.

  deprecated_since_version="1.5"


Successful time-scale separation solution obtained.
the ATP error is: 1377.5052417105946


In [6]:
p = bokeh.plotting.figure(width = 475, height = 350, title = 'Reduced Model 5',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
#colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
colors = colorcet.b_glasbey_category10
p.line(timepoints_ode, sol[:,1],line_width = 2, legend_label = 'atp', color = colors[3])
p.line(timepoints_ode, sol[:,2], line_width = 2,legend_label = 'glucose', color = colors[0])
p.line(timepoints_ode, sol[:,34], line_width = 2,legend_label = 'isobutanol', color = colors[1])
p.legend.click_policy = 'hide'
bokeh.io.show(p)

## Functions 
---

In [43]:
aa.f

[-enzyme_hex*metabolite_adp*molecule_g6p*r3_1_k + enzyme_hex*metabolite_atp*molecule_glucose*r0_1_k*r0_k/(r0_1_k + r2_k) - enzyme_hex*metabolite_atp*molecule_glucose*r0_k + enzyme_hex*(metabolite_adp*molecule_g6p*r0_1_k*r3_1_k + metabolite_adp*molecule_g6p*r2_k*r3_1_k + metabolite_atp*molecule_glucose*r0_k*r2_k)/(r0_1_k + r2_k),
 complex_enzyme_atpase_metabolite_atp*r75_1_k - enzyme_atpase*metabolite_atp*r75_k + enzyme_hex*metabolite_atp*molecule_glucose*r0_1_k*r0_k/(r0_1_k + r2_k) - enzyme_hex*metabolite_atp*molecule_glucose*r0_k + enzyme_pfk*metabolite_atp*molecule_f6p*r10_1_k*r10_k/(r10_1_k + r12_k) - enzyme_pfk*metabolite_atp*molecule_f6p*r10_k - enzyme_pgk*metabolite_atp*r33_1_k*pow(molecule_3pg, 2) + enzyme_pgk*(metabolite_adp*molecule_13bpg*r30_k*r32_k + metabolite_atp*r30_1_k*r33_1_k*pow(molecule_3pg, 2) + metabolite_atp*r32_k*r33_1_k*pow(molecule_3pg, 2))/(r30_1_k + r32_k) - 2.0*enzyme_pyk*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2) + 2.0*enzyme_pyk*(r45_1_k*r48_1

In [ ]:
aa.f

In [ ]:
ok = str(aa.f[0])
ok

#### If a + two parameters is detected, get start and end indices and add to a list of possible lumped parameters

In [7]:
# This function to find all particular substrings in a string (copied from internet)
def find_all(a_str, sub):
    '''Get all instances of a particular substring in a string'''
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to 


In [8]:
# This function to find all particular substrings in a string (copied from internet)
def find_all(a_str, sub):
    '''Get all instances of a particular substring in a string'''
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to 


def find_possible_lumped_params(curr_str, possible_param_str):
    '''Get all possible lumped parameters in a particular string'''
    possible_lumped_params = []
    indices_list = list(find_all(curr_str, possible_param_str))

    for start_index in indices_list:
        # get index of first r
        r_ind = curr_str.find('r', start_index - 7, start_index)
        # get index of last k
        k_ind = curr_str.find('k', start_index + 4, start_index + 4 + 7)
        possible_lumped_params.append(curr_str[r_ind : k_ind + 1])
        
    return list(set(possible_lumped_params))
        

In [9]:
find_possible_lumped_params(ok, 'k + r')

NameError: name 'ok' is not defined

In [12]:
# Get all possible lumped parameters in all the functions
all_possible_lumped_params = []
for i,_ in enumerate(aa.f):

    curr_str = str(aa.f[i])
    #print(curr_str)
    temp_list = find_possible_lumped_params(curr_str, 'k + r')
    #print(temp_list)
    for temp in temp_list:
        #print(all_possible_lumped_params)
        all_possible_lumped_params.append(temp)
        


In [ ]:
#all_possible_lumped_params

In [11]:
from collections import Counter

In [13]:
counts = Counter(all_possible_lumped_params)
counts

Counter({'r0_1_k + r2_k': 5,
         'r30_1_k + r32_k': 5,
         'r45_1_k + r47_k': 5,
         'r10_1_k + r12_k': 5,
         'r5_1_k + r7_k': 3,
         'r15_1_k + r17_k': 3,
         'r17_k + r15_1_k': 2,
         'r20_1_k + r22_k': 5,
         'r25_1_k + r27_k': 6,
         'r70_1_k + r72_k': 5,
         'r55_1_k + r57_k': 5,
         'r35_1_k + r37_k': 3,
         'r40_1_k + r42_k': 3,
         'r50_1_k + r52_k': 3,
         'r53_1_k + r50_k': 2,
         'r60_1_k + r62_k': 3,
         'r65_1_k + r67_k': 3})

In [14]:
counts_list = list(counts)

Anything that shows up more than 2 times can be lumped:

In [15]:
# Remove anything with 2
counts_list.remove('r17_k + r15_1_k')
counts_list.remove('r53_1_k + r50_k')
counts_list

['r0_1_k + r2_k',
 'r30_1_k + r32_k',
 'r45_1_k + r47_k',
 'r10_1_k + r12_k',
 'r5_1_k + r7_k',
 'r15_1_k + r17_k',
 'r20_1_k + r22_k',
 'r25_1_k + r27_k',
 'r70_1_k + r72_k',
 'r55_1_k + r57_k',
 'r35_1_k + r37_k',
 'r40_1_k + r42_k',
 'r50_1_k + r52_k',
 'r60_1_k + r62_k',
 'r65_1_k + r67_k']

In [40]:
# Make a dictionary with keys = counts_list, values = name of lumped parameter
lumped_dict = {}
alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o',]# 'ba', 'bb']

for j,item in enumerate(counts_list):
    lumped_dict[item] = alpha[j]
    
lumped_dict

{'r0_1_k + r2_k': 'a',
 'r30_1_k + r32_k': 'b',
 'r45_1_k + r47_k': 'c',
 'r10_1_k + r12_k': 'd',
 'r5_1_k + r7_k': 'e',
 'r15_1_k + r17_k': 'f',
 'r20_1_k + r22_k': 'g',
 'r25_1_k + r27_k': 'h',
 'r70_1_k + r72_k': 'i',
 'r55_1_k + r57_k': 'j',
 'r35_1_k + r37_k': 'k',
 'r40_1_k + r42_k': 'l',
 'r50_1_k + r52_k': 'm',
 'r60_1_k + r62_k': 'n',
 'r65_1_k + r67_k': 'o'}

In [17]:
# Replace every instance of the keys (in aa.f) with the appropriate values
new_funcs = []
for i,_ in enumerate(aa.f):
    curr_str = str(aa.f[i])
    for j,replace in enumerate(counts_list):
        if replace in curr_str:
            #print(replace, alpha[j])
            curr_str = curr_str.replace(replace,alpha[j])
            #print(curr_str)
            #new_funcs.append(curr_str)
    new_funcs.append(curr_str)
            
            #new_funcs.append(curr_str)
            
    
        

In [18]:
new_funcs[0]

'-enzyme_hex*metabolite_adp*molecule_g6p*r3_1_k + enzyme_hex*metabolite_atp*molecule_glucose*r0_1_k*r0_k/(a) - enzyme_hex*metabolite_atp*molecule_glucose*r0_k + enzyme_hex*(metabolite_adp*molecule_g6p*r0_1_k*r3_1_k + metabolite_adp*molecule_g6p*r2_k*r3_1_k + metabolite_atp*molecule_glucose*r0_k*r2_k)/(a)'

#### Get for any k*r item

In [19]:
# Get all possible lumped parameters in all the functions
all_possible_lumped_params2 = []
for i,_ in enumerate(new_funcs):

    curr_str = str(new_funcs[i])
    #print(curr_str)
    temp_list = find_possible_lumped_params(curr_str, 'k*r')
    #print(temp_list)
    for temp in temp_list:
        #print(all_possible_lumped_params)
        all_possible_lumped_params2.append(temp)
        



In [20]:
counts2 = Counter(all_possible_lumped_params2)

In [21]:
# Remove anything with 2
counts_list2 = list(counts2)
counts_list2.remove('r5_1_k*r5_k')
counts_list2.remove('r7_k*r8_1_k')
counts_list2.remove('r5_1_k*r8_1_k')
counts_list2.remove('r5_k*r7_k')
counts_list2.remove('r15_1_k*r15_k')
counts_list2.remove('r17_k*r18_1_k')
counts_list2.remove('r15_k*r17_k')
counts_list2.remove('r15_1_k*r18_1_k')
counts_list2.remove('r35_1_k*r35_k')
counts_list2.remove('r35_k*r37_k')
counts_list2.remove('r35_1_k*r38_1_k')
counts_list2.remove('r37_k*r38_1_k')
counts_list2.remove('r40_1_k*r40_k')
counts_list2.remove('r40_1_k*r43_1_k')
counts_list2.remove('r42_k*r43_1_k')
counts_list2.remove('r40_k*r42_k')
counts_list2.remove('r50_1_k*r50_k')
counts_list2.remove('r50_1_k*r53_1_k')
counts_list2.remove('r50_k*r52_k')
counts_list2.remove('r52_k*r53_1_k')
counts_list2.remove('r60_1_k*r60_k')
counts_list2.remove('r62_k*r63_1_k')
counts_list2.remove('r60_1_k*r63_1_k')
counts_list2.remove('r60_k*r62_k')
counts_list2.remove('r65_1_k*r65_k')
counts_list2.remove('r65_1_k*r68_1_k')
counts_list2.remove('r65_k*r67_k')
counts_list2.remove('r67_k*r68_1_k')
counts_list2.remove('')


In [22]:
counts_list2

['r0_k*r2_k',
 'r0_1_k*r0_k',
 'r2_k*r3_1_k',
 'r0_1_k*r3_1_k',
 'r32_k*r33_1_k',
 'r45_1_k*r48_1_k',
 'r45_k*r47_k',
 'r10_1_k*r10_k',
 'r30_1_k*r33_1_k',
 'r30_k*r32_k',
 'r47_k*r48_1_k',
 'r12_k*r13_1_k',
 'r10_k*r12_k',
 'r30_1_k*r30_k',
 'r45_1_k*r45_k',
 'r10_1_k*r13_1_k',
 'r20_1_k*r20_k',
 'r25_1_k*r25_k',
 'r22_k*r23_1_k',
 'r20_1_k*r23_1_k',
 'r20_k*r22_k',
 'r57_k*r58_1_k',
 'r70_1_k*r73_1_k',
 'r55_k*r57_k',
 'r72_k*r73_1_k',
 'r55_1_k*r58_1_k',
 'r70_k*r72_k',
 'r25_k*r27_k',
 'r27_k*r28_1_k',
 'r55_1_k*r55_k',
 'r70_1_k*r70_k',
 'r25_1_k*r28_1_k']

In [23]:
len(counts_list2)

32

In [27]:
alpha2 = ['p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'aa', 'ab', 'ac', 'ad', 'ae', 'af', 'ag', 'ah',
'ai', 'aj', 'ak', 'al', 'am', 'an', 'ao', 'ap', 'aq', 'ar', 'as', 'at', 'au']# 'av', 'aw', 'ax', 'ay', 'az',
#          'bc', 'bd', 'be', 'bf', 'bg', 'bh', 'bi', 'bj', 'bk', 'bl', 'bm', 'bn', 'bo', 'bp', 'bq', 'br',
#          'bs', 'bt', 'bu']

In [28]:
len(alpha2)

32

In [29]:
lumped_dict2 = {}
for j,item in enumerate(counts_list2):
    lumped_dict2[item] = alpha2[j]
    
lumped_dict2

{'r0_k*r2_k': 'p',
 'r0_1_k*r0_k': 'q',
 'r2_k*r3_1_k': 'r',
 'r0_1_k*r3_1_k': 's',
 'r32_k*r33_1_k': 't',
 'r45_1_k*r48_1_k': 'u',
 'r45_k*r47_k': 'v',
 'r10_1_k*r10_k': 'w',
 'r30_1_k*r33_1_k': 'x',
 'r30_k*r32_k': 'y',
 'r47_k*r48_1_k': 'z',
 'r12_k*r13_1_k': 'aa',
 'r10_k*r12_k': 'ab',
 'r30_1_k*r30_k': 'ac',
 'r45_1_k*r45_k': 'ad',
 'r10_1_k*r13_1_k': 'ae',
 'r20_1_k*r20_k': 'af',
 'r25_1_k*r25_k': 'ag',
 'r22_k*r23_1_k': 'ah',
 'r20_1_k*r23_1_k': 'ai',
 'r20_k*r22_k': 'aj',
 'r57_k*r58_1_k': 'ak',
 'r70_1_k*r73_1_k': 'al',
 'r55_k*r57_k': 'am',
 'r72_k*r73_1_k': 'an',
 'r55_1_k*r58_1_k': 'ao',
 'r70_k*r72_k': 'ap',
 'r25_k*r27_k': 'aq',
 'r27_k*r28_1_k': 'ar',
 'r55_1_k*r55_k': 'as',
 'r70_1_k*r70_k': 'at',
 'r25_1_k*r28_1_k': 'au'}

In [30]:
# Replace every instance of the keys (in aa.f) with the appropriate values
new_funcs2 = []
for i,_ in enumerate(new_funcs):
    curr_str = str(new_funcs[i])
    for j,replace in enumerate(counts_list2):
        if replace in curr_str:
            #print(replace, alpha[j])
            curr_str = curr_str.replace(replace,alpha2[j])
            #print(curr_str)
            #new_funcs.append(curr_str)
    new_funcs2.append(curr_str)
            
            #new_funcs.append(curr_str)
            
    
        

In [51]:
new_funcs2[7]

'-enzyme_pfk*metabolite_adp*molecule_f16p*r13_1_k + enzyme_pfk*metabolite_atp*molecule_f6p*w/(d) - enzyme_pfk*metabolite_atp*molecule_f6p*r10_k + enzyme_pfk*(metabolite_adp*molecule_f16p*ae + metabolite_adp*molecule_f16p*aa + metabolite_atp*molecule_f6p*ab)/(d)'

## Let's count how many parameters we have now

In [41]:
new_params_list = alpha + alpha2

In [33]:
new_funcs2[10]

'-2.0*enzyme_ald_tpi*r18_1_k*pow(molecule_g3p, 2) + 2.0*enzyme_ald_tpi*(molecule_f16p*r15_k*r17_k + r15_1_k*r18_1_k*pow(molecule_g3p, 2) + r17_k*r18_1_k*pow(molecule_g3p, 2))/(f) + 2.0*enzyme_gapN*af*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2)/(g) - 2.0*enzyme_gapN*r20_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2) + 2.0*enzyme_mGapDH*metabolite_pi*ag*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2)/(h) - 2.0*enzyme_mGapDH*metabolite_pi*r25_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2)'

In [34]:
def find_remaining_params(curr_str, search_str):
    ret_list = []
    
    indices_list = list(find_all(curr_str, search_str))
    #print(indices_list)
    
    for start_index in indices_list:
        # get index of last k
        k_ind = curr_str.find('k', start_index, start_index + 8)
        
        # get index of first r
        r_ind = curr_str.find('r', start_index, start_index + 3)
        #print(k_ind)
        list_add = curr_str[r_ind : k_ind + 1]
        
        
        ret_list.append(list_add)
        
    if '' in ret_list:
        ret_list.remove('')
    
    
    return list(set(ret_list))

In [35]:
# test
find_remaining_params(new_funcs2[22], '*r')
# find_remaining_params(new_funcs2[20], '(r')
# find_remaining_params(new_funcs2[22], '+ r')

['r42_k', 'r45_k', 'r43_1_k']

In [42]:
params_now = []
for func in new_funcs2:
    for item in find_remaining_params(func, '*r'):
        params_now.append(item)
    for item in find_remaining_params(func, '(r'):
        params_now.append(item)
    for item in find_remaining_params(func, '+ r'):
        params_now.append(item)
    
    
#     params_now.append(find_remaining_params(func, '*r'))
#     params_now.append(find_remaining_params(func, '(r'))
#     params_now.append(find_remaining_params(func, '+ r'))
    
print('the current num of params is:',len(list(set(params_now))) + len(new_params_list))

the current num of params is: 96


### Get old num of params

In [37]:
%run sbml_to_ode2.py

In [38]:
x,f,P,params_values,x_init = sbml_to_ode2('CRN.xml')

In [39]:
print('the old num of params was:', len(params_values))

the old num of params was: 80


In [ ]:
new_funcs2

# Noooo! In this notebook we got the params from 80 to 96 ripppp

In [ ]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,libsbml,sys,auto_reduce,colorcet,jupyterlab